In [1]:
#import all required libraries

from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np

website='https://www.theverge.com/'
response = requests.get(website)

soup = BeautifulSoup(response.content, 'html.parser')
df=pd.read_csv("ddmmyyy_verge.csv")

In [2]:
# get all links of articles

toplink = soup.find_all('a',{'class' : 'group-hover:shadow-highlight-blurple'})
links = soup.find_all('a', {'class': 'group-hover:shadow-underline-franklin'})
all_links=[]

for i in toplink:
    all_links.append('https://www.theverge.com'+i['href'])
for link in links:
    (all_links.append('https://www.theverge.com'+link['href']))

In [3]:
# get all headline of the article

topheadline = soup.find_all('a',{'class' : 'group-hover:shadow-highlight-blurple'})
headline = soup.find_all('a', {'class': 'group-hover:shadow-underline-franklin'})
headlines=[]

for i in topheadline:
    headlines.append(i.text)
for line in headline:
    (headlines.append(line.text))

In [4]:
# to get all dates of every article

new_dates=[]
dates=[]
for i in all_links:
    response = requests.get(i)
    soup = BeautifulSoup(response.content, 'html.parser')
    date = soup.find('time',{'class' : 'duet--article--timestamp font-polysans text-12'})
    try:
        new_dates.append(date.text)
    except:
        dates.append(0)
        pass

for x in new_dates:
    try:
        if x.strip():
            dates.append(x.strip())
    except:
        pass
try:
    dates.remove("Updated")
except:
    pass

[0,
 0,
 'Apr 8, 2023, 1:00 PM UTC',
 'Apr 8, 2023, 12:30 PM UTC',
 'Apr 8, 2023, 4:00 PM UTC',
 'Apr 7, 2023, 8:35 PM UTC']

In [5]:
#to get authors of all articles

authors=[]
website='https://www.theverge.com/'
response = requests.get(website)
soup = BeautifulSoup(response.content, 'html.parser')
author = soup.find_all('a', {'class': 'text-gray-31 hover:shadow-underline-inherit dark:text-franklin mr-8'})

try:
    for name in author:
        (authors.append(name.text))
except:
    authors.append("")


In [6]:
#to create a unique id

ids=[]
for i in headlines:
    ids.append(i[:3]+str(len(i)))


In [7]:
# create a dataframe with all the extracted data

df1=pd.DataFrame({'id':ids,'url':all_links,'headline':headlines,'author':authors,'date':dates})

In [8]:
# Convert the string to datetime format

df1['date'] = pd.to_datetime(df1['date'])

# Extract only the date part
df1['date'] = df1['date'].dt.strftime('%Y/%m/%d')



In [9]:
# craete a final dataset by removing duplicate rows and joining csv dataframe 

result=pd.concat([df,df1],axis=0)
result.dropna(how='all', axis=1, inplace=True)
result.drop(columns='id',inplace=True)
df=result
df.drop_duplicates(inplace=True)


In [10]:
# set index column as id

df.index = range(0, len(df))
df.index.name = 'id'


In [11]:
#append data from dataframe into csv

df.to_csv('ddmmyyy_verge.csv', mode='w', header=True,index=True,index_label='id')

In [12]:
#creating a sqlite database

import sqlite3
import csv

# Create a connection object
conn = sqlite3.connect('assessment.db')

# Create a cursor object
c = conn.cursor()

#create table if not there
try:
    c.execute('CREATE TABLE ddmmyyy_verge(id INT(6) PRIMARY KEY,url VARCHAR(255),headline VARCHAR(255),author VARCHAR(255),date VARCHAR(255))')
except:
    pass

#append table with data from dataframe
try:
    df.to_sql('ddmmyyy_verge', conn, if_exists='append', index=True)
except:
    pass
# Commit the changes
conn.commit()
                                                                                                                           
# Close the connection
conn.close()